In [18]:
import pandas as pd
import numpy as np
import csv
from pathlib import Path

# Archivos a revisar
archivos = [
    "calama_corrupted.csv",
    "salvador_corrupted.csv",
    "Vallenar_corrupted.csv"
]

# Rango físico válido por columna
rangos_fisicos = {
    "GHI": (0, 1200),
    "DNI": (0, 1000),
    "DHI": (0, 600),
    "TempC": (-40, 60),
    "Wind_mps": (0, 40),
    "Pressure": (500, 1100),  # si existe
}

def revisar_csv(ruta_archivo):
    print(f"\n📂 Analizando: {ruta_archivo}")
    ruta = Path(ruta_archivo)

    if not ruta.exists():
        print(f"❌ Archivo no encontrado.")
        return

    # Contar filas con número incorrecto de columnas
    with open(ruta, 'r', newline='', encoding='utf-8') as f:
        reader = csv.reader(f)
        header = next(reader)
        num_columnas = len(header)
        filas_malas = 0
        for fila in reader:
            if len(fila) != num_columnas:
                filas_malas += 1

    print(f"📌 Columnas esperadas: {num_columnas}")
    print(f"❗ Filas con número incorrecto de columnas: {filas_malas}")

    # Leer como DataFrame e intentar analizar
    try:
        df = pd.read_csv(ruta)
    except Exception as e:
        print(f"❌ Error al leer el archivo como DataFrame: {e}")
        return

    print(f"✅ Archivo leído correctamente. Registros: {len(df)}")

    # Valores faltantes
    nulos = df.isnull().sum()
    total_nulos = nulos[nulos > 0]
    if not total_nulos.empty:
        print("\n🔎 Valores faltantes por columna:")
        print(total_nulos)
    else:
        print("\n✅ No hay valores faltantes.")

    # Valores fuera de rango
    for col, (vmin, vmax) in rangos_fisicos.items():
        if col in df.columns:
            negativos = (df[col] < 0).sum() if vmin >= 0 else 0
            fuera_rango = ((df[col] < vmin) | (df[col] > vmax)).sum()
            if fuera_rango > 0:
                print(f"⚠️ '{col}': {fuera_rango} valores fuera del rango [{vmin}, {vmax}]")
            elif negativos > 0:
                print(f"⚠️ '{col}': {negativos} valores negativos (no válidos)")
            else:
                print(f"✅ '{col}' dentro de los límites físicos.")

    # Estadísticas generales
    print("\n📊 Estadísticas generales:")
    print(df.describe(include='all').transpose().round(2))


# Ejecutar análisis por archivo
for archivo in archivos:
    revisar_csv(archivo)



📂 Analizando: calama_corrupted.csv
📌 Columnas esperadas: 15
❗ Filas con número incorrecto de columnas: 0
✅ Archivo leído correctamente. Registros: 8760

🔎 Valores faltantes por columna:
GHI           180
DNI           163
DHI           148
Tdry          174
Tdew          133
RH            172
Pres          153
Wspd          172
Wdir          179
Snow Depth    140
dtype: int64
⚠️ 'GHI': 374 valores fuera del rango [0, 1200]
⚠️ 'DNI': 2857 valores fuera del rango [0, 1000]
⚠️ 'DHI': 318 valores fuera del rango [0, 600]

📊 Estadísticas generales:
             count     mean     std      min      25%     50%      75%  \
Year        8760.0  2008.66    3.60  2004.00  2005.00  2008.0  2012.00   
Month       8760.0     6.53    3.45     1.00     4.00     7.0    10.00   
Day         8760.0    15.72    8.80     1.00     8.00    16.0    23.00   
Hour        8760.0    11.50    6.92     0.00     5.75    11.5    17.25   
Minute      8760.0     0.00    0.00     0.00     0.00     0.0     0.00   
GHI  